# Image Extractor - Standalone Pipeline

Extract images, charts, and diagrams from PDFs using **precise detection** + Vision LLM descriptions.

**Features:**
- **PyMuPDF**: Extracts embedded images with exact boundaries (FREE)
- **LayoutParser**: Detects figure/chart regions that aren't embedded images (FREE)
- **GPT-4o Vision**: Describes extracted images only (cost-efficient)
- Outputs `image_extractions.json`

**No Google Doc AI dependencies. Precise bounding boxes. Minimal LLM cost.**

---

## How It Works

1. **PyMuPDF**: Extract all embedded images (photos, scans) with exact coordinates
2. **LayoutParser**: Detect "Figure" regions on rendered pages (vector charts, infographics)
3. **Combine**: Merge both sources, deduplicate overlapping regions
4. **Describe**: Send ONLY extracted images to GPT-4o (not full pages)
5. **Output**: Save JSON with image metadata + descriptions

---

## Cost Comparison

| Approach | What Goes to LLM | Cost (10-page PDF) |
|----------|-----------------|-------------------|
| Old (full page) | Every page | ~$0.30 |
| New (images only) | Only 5 images | ~$0.05 |


## 1. Install Dependencies


In [ ]:
# Install Python packages
%pip install -q pdf2image Pillow openai PyMuPDF

# Install LayoutParser with detection models
%pip install -q layoutparser
%pip install -q 'layoutparser[layoutmodels]' torch torchvision

# Install Poppler (required by pdf2image to render PDFs)
!apt-get update -qq && apt-get install -y -qq poppler-utils

print("✅ Dependencies installed successfully!")
print("   - PyMuPDF: Extract embedded images")
print("   - LayoutParser: Detect figure regions")
print("   - OpenAI: Image descriptions")


## 2. Configuration


In [ ]:
import os

# ============================================================
# CONFIGURATION - Update these values
# ============================================================

# OpenAI API Key (for Vision LLM)
OPENAI_API_KEY = ""  # Paste your key here, or set via environment

# Output directory for extracted images
OUTPUT_DIR = "image_output"

# ============================================================
# Set environment variable
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
elif not os.getenv("OPENAI_API_KEY"):
    print("⚠️ Warning: OPENAI_API_KEY not set. Please set it above or via environment.")

# Create output directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "images"), exist_ok=True)

print(f"✅ Output directory: {OUTPUT_DIR}/")
print(f"✅ Images will be saved to: {OUTPUT_DIR}/images/")


## 3. Upload PDF


In [ ]:
from google.colab import files

print("Upload a PDF file to extract images from...")
uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]
print(f"✅ Uploaded: {pdf_filename}")


## 4. Image Extractor Class

**Hybrid detection pipeline:**
1. PyMuPDF → Extracts embedded images (exact boundaries)
2. LayoutParser → Detects figure regions (vector charts, infographics)
3. GPT-4o → Describes each extracted image (cost-efficient)


In [ ]:
import json
import base64
from typing import List, Dict, Any, Optional, Tuple
from pdf2image import convert_from_path
from PIL import Image
import numpy as np
import io
import fitz  # PyMuPDF
from openai import OpenAI
from datetime import datetime

# Import LayoutParser (with fallback if not available)
try:
    import layoutparser as lp
    LAYOUTPARSER_AVAILABLE = True
    print("✅ LayoutParser loaded")
except ImportError:
    LAYOUTPARSER_AVAILABLE = False
    print("⚠️ LayoutParser not available - will use PyMuPDF only")


class ImageExtractor:
    """
    Extracts images, charts, and diagrams from PDFs using hybrid detection.
    
    Pipeline:
    1. PyMuPDF: Extract embedded images (photos, scans) - exact boundaries
    2. LayoutParser: Detect figure regions (vector charts, infographics) - ML detection
    3. Combine: Merge both sources, deduplicate overlapping regions
    4. GPT-4o: Describe each extracted image (NOT full pages - cost efficient)
    """
    
    def __init__(self, output_dir: str = "image_output", api_key: Optional[str] = None,
                 use_layoutparser: bool = True, generate_descriptions: bool = True):
        """
        Args:
            output_dir: Directory to save extracted images
            api_key: OpenAI API key (uses env var if not provided)
            use_layoutparser: Whether to use LayoutParser for figure detection
            generate_descriptions: Whether to generate LLM descriptions (costs money)
        """
        self.output_dir = output_dir
        self.images_dir = os.path.join(output_dir, "images")
        self.use_layoutparser = use_layoutparser and LAYOUTPARSER_AVAILABLE
        self.generate_descriptions = generate_descriptions
        
        # Create directories
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.images_dir, exist_ok=True)
        
        # Initialize OpenAI client (only if generating descriptions)
        self.client = None
        if generate_descriptions:
            api_key = api_key or os.getenv("OPENAI_API_KEY")
            if api_key:
                self.client = OpenAI(api_key=api_key)
            else:
                print("⚠️ No API key - descriptions will be skipped")
                self.generate_descriptions = False
        
        # Initialize LayoutParser model
        self.lp_model = None
        if self.use_layoutparser:
            try:
                # PubLayNet model - detects Text, Title, List, Table, Figure
                self.lp_model = lp.Detectron2LayoutModel(
                    'lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
                    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5],
                    label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"}
                )
                print("✅ LayoutParser model loaded")
            except Exception as e:
                print(f"⚠️ Could not load LayoutParser model: {e}")
                self.use_layoutparser = False
        
        # Counter for image IDs
        self.image_counter = 0
    
    def extract(self, pdf_path: str) -> Dict[str, Any]:
        """
        Main entry point - extract all images from a PDF.
        
        Returns:
            Dictionary with metadata and list of extracted images
        """
        print(f"📄 Processing: {pdf_path}")
        
        all_images = []
        
        # 1. Extract embedded images using PyMuPDF
        print("\n🔍 Step 1: Extracting embedded images (PyMuPDF)...")
        embedded_images = self._extract_embedded_images(pdf_path)
        print(f"   Found {len(embedded_images)} embedded images")
        all_images.extend(embedded_images)
        
        # 2. Detect figure regions using LayoutParser
        if self.use_layoutparser:
            print("\n🔍 Step 2: Detecting figure regions (LayoutParser)...")
            figure_regions = self._detect_figure_regions(pdf_path)
            print(f"   Found {len(figure_regions)} figure regions")
            
            # Deduplicate - remove figures that overlap with embedded images
            unique_figures = self._deduplicate_regions(figure_regions, embedded_images)
            print(f"   After deduplication: {len(unique_figures)} unique figures")
            all_images.extend(unique_figures)
        else:
            print("\n⏭️ Step 2: Skipping LayoutParser (not available)")
        
        # 3. Generate descriptions for each image
        if self.generate_descriptions and self.client:
            print(f"\n🤖 Step 3: Generating descriptions for {len(all_images)} images...")
            for i, img_data in enumerate(all_images):
                print(f"   Describing image {i+1}/{len(all_images)}...")
                if img_data.get("file_path") and os.path.exists(img_data["file_path"]):
                    description = self._describe_image(img_data["file_path"])
                    img_data["description"] = description
        else:
            print("\n⏭️ Step 3: Skipping descriptions (disabled or no API key)")
        
        # 4. Get page count
        doc = fitz.open(pdf_path)
        page_count = len(doc)
        doc.close()
        
        # 5. Build result
        result = {
            "metadata": {
                "source_file": os.path.basename(pdf_path),
                "page_count": page_count,
                "images_found": len(all_images),
                "extraction_timestamp": datetime.now().isoformat(),
                "methods_used": {
                    "pymupdf": True,
                    "layoutparser": self.use_layoutparser,
                    "descriptions": self.generate_descriptions
                }
            },
            "images": all_images
        }
        
        print(f"\n✅ Extraction complete! Found {len(all_images)} images across {page_count} pages.")
        return result
    
    def _extract_embedded_images(self, pdf_path: str) -> List[Dict[str, Any]]:
        """
        Extract embedded images from PDF using PyMuPDF.
        These are actual image files (JPG/PNG) stored inside the PDF.
        """
        extracted = []
        doc = fitz.open(pdf_path)
        
        for page_num in range(len(doc)):
            page = doc[page_num]
            image_list = page.get_images(full=True)
            
            for img_index, img_info in enumerate(image_list):
                xref = img_info[0]  # Image reference number
                
                try:
                    # Extract the image
                    base_image = doc.extract_image(xref)
                    image_bytes = base_image["image"]
                    image_ext = base_image["ext"]
                    
                    # Get image position on page
                    img_rect = page.get_image_rects(xref)
                    bbox = None
                    if img_rect:
                        rect = img_rect[0]  # First occurrence
                        page_rect = page.rect
                        # Normalize to 0-1 range
                        bbox = [
                            rect.x0 / page_rect.width,
                            rect.y0 / page_rect.height,
                            rect.x1 / page_rect.width,
                            rect.y1 / page_rect.height
                        ]
                    
                    # Save image
                    self.image_counter += 1
                    image_id = f"img_{self.image_counter:03d}"
                    filename = f"page{page_num + 1}_{image_id}.{image_ext}"
                    save_path = os.path.join(self.images_dir, filename)
                    
                    with open(save_path, "wb") as f:
                        f.write(image_bytes)
                    
                    extracted.append({
                        "id": image_id,
                        "page": page_num + 1,
                        "type": "embedded",
                        "source": "pymupdf",
                        "bbox": bbox,
                        "file_path": save_path,
                        "description": ""
                    })
                    
                except Exception as e:
                    print(f"   ⚠️ Error extracting image {xref} on page {page_num + 1}: {e}")
        
        doc.close()
        return extracted
    
    def _detect_figure_regions(self, pdf_path: str) -> List[Dict[str, Any]]:
        """
        Detect figure regions using LayoutParser.
        Catches vector graphics, charts, and diagrams that aren't embedded images.
        """
        if not self.lp_model:
            return []
        
        detected = []
        
        # Render pages as images for LayoutParser
        page_images = convert_from_path(pdf_path, dpi=150)
        
        for page_num, page_image in enumerate(page_images):
            # Convert PIL to numpy array for LayoutParser
            page_array = np.array(page_image)
            
            # Detect layout
            layout = self.lp_model.detect(page_array)
            
            # Filter for Figure blocks only
            figures = [block for block in layout if block.type == "Figure"]
            
            for fig in figures:
                # Get bounding box (normalized)
                width, height = page_image.size
                bbox = [
                    fig.block.x_1 / width,
                    fig.block.y_1 / height,
                    fig.block.x_2 / width,
                    fig.block.y_2 / height
                ]
                
                # Crop and save
                self.image_counter += 1
                image_id = f"img_{self.image_counter:03d}"
                
                # Crop the figure region
                crop_box = (
                    int(fig.block.x_1),
                    int(fig.block.y_1),
                    int(fig.block.x_2),
                    int(fig.block.y_2)
                )
                cropped = page_image.crop(crop_box)
                
                filename = f"page{page_num + 1}_{image_id}.png"
                save_path = os.path.join(self.images_dir, filename)
                cropped.save(save_path, format='PNG')
                
                detected.append({
                    "id": image_id,
                    "page": page_num + 1,
                    "type": "figure",
                    "source": "layoutparser",
                    "bbox": bbox,
                    "file_path": save_path,
                    "description": ""
                })
        
        return detected
    
    def _deduplicate_regions(self, figures: List[Dict], embedded: List[Dict]) -> List[Dict]:
        """
        Remove figure regions that significantly overlap with embedded images.
        This prevents duplicate extraction of the same visual.
        """
        unique = []
        
        for fig in figures:
            fig_bbox = fig.get("bbox")
            fig_page = fig.get("page")
            
            if not fig_bbox:
                unique.append(fig)
                continue
            
            # Check overlap with embedded images on same page
            is_duplicate = False
            for emb in embedded:
                if emb.get("page") != fig_page:
                    continue
                
                emb_bbox = emb.get("bbox")
                if not emb_bbox:
                    continue
                
                # Calculate IoU (Intersection over Union)
                iou = self._calculate_iou(fig_bbox, emb_bbox)
                if iou > 0.5:  # More than 50% overlap = duplicate
                    is_duplicate = True
                    break
            
            if not is_duplicate:
                unique.append(fig)
        
        return unique
    
    def _calculate_iou(self, box1: List[float], box2: List[float]) -> float:
        """Calculate Intersection over Union of two bounding boxes."""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 <= x1 or y2 <= y1:
            return 0.0
        
        intersection = (x2 - x1) * (y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0
    
    def _describe_image(self, image_path: str) -> str:
        """
        Send a single extracted image to GPT-4o for description.
        Only the image goes to the LLM - NOT the full page.
        """
        if not self.client:
            return ""
        
        try:
            # Read and encode image
            with open(image_path, "rb") as f:
                image_bytes = f.read()
            base64_image = base64.b64encode(image_bytes).decode('utf-8')
            
            # Simple prompt for description only
            prompt = """Describe this image in 1-2 sentences. Focus on:
- What type of visual it is (chart, diagram, photo, etc.)
- The key information or data it shows
- Any labels or important text visible

Be concise and factual."""

            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/png;base64,{base64_image}",
                                    "detail": "low"  # Lower cost for descriptions
                                }
                            }
                        ]
                    }
                ],
                max_tokens=150,
                temperature=0.2
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"   ⚠️ Error describing image: {e}")
            return ""


print("✅ ImageExtractor class defined (PyMuPDF + LayoutParser + GPT-4o)")


## 5. Run Extraction


In [ ]:
# Initialize extractor and run
# Options:
#   use_layoutparser=True  - Detect figure regions (vector charts, infographics)
#   generate_descriptions=True - Generate GPT-4o descriptions (costs money)

extractor = ImageExtractor(
    output_dir=OUTPUT_DIR,
    use_layoutparser=True,       # Set to False to use only PyMuPDF
    generate_descriptions=True   # Set to False to skip LLM descriptions
)
result = extractor.extract(pdf_filename)

# Preview result summary
print("\n" + "="*60)
print("EXTRACTION SUMMARY")
print("="*60)
print(f"Source: {result['metadata']['source_file']}")
print(f"Pages: {result['metadata']['page_count']}")
print(f"Images Found: {result['metadata']['images_found']}")
print(f"Methods: PyMuPDF={result['metadata']['methods_used']['pymupdf']}, "
      f"LayoutParser={result['metadata']['methods_used']['layoutparser']}, "
      f"Descriptions={result['metadata']['methods_used']['descriptions']}")
print("="*60)


## 6. Preview Extracted Images


In [ ]:
from IPython.display import display, Image as IPImage

if result["images"]:
    print(f"Showing {len(result['images'])} extracted images:\n")
    
    for img_data in result["images"]:
        print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        source_icon = "📦" if img_data.get('source') == 'pymupdf' else "🔍"
        print(f"{source_icon} {img_data['id']} | Page {img_data['page']} | Type: {img_data['type']} | Source: {img_data.get('source', 'unknown')}")
        if img_data.get('description'):
            print(f"📝 {img_data['description']}")
        print(f"📁 {img_data['file_path']}")
        
        # Display the image
        if img_data.get('file_path') and os.path.exists(img_data['file_path']):
            display(IPImage(filename=img_data['file_path'], width=400))
        print()
else:
    print("No images were detected in this PDF.")
    print("\nPossible reasons:")
    print("  - PDF has no embedded images (try enabling LayoutParser)")
    print("  - Images are vector graphics (LayoutParser may detect these)")
    print("  - PDF is scanned text only")


## 7. Save JSON Output


In [ ]:
# Save JSON output
output_json_path = os.path.join(OUTPUT_DIR, "image_extractions.json")

with open(output_json_path, "w") as f:
    json.dump(result, f, indent=2)

print(f"✅ JSON saved to: {output_json_path}")

# Display JSON content
print("\n📄 JSON Content Preview:")
print("-" * 50)
print(json.dumps(result, indent=2)[:2000])
if len(json.dumps(result, indent=2)) > 2000:
    print("\n... (truncated)")


## 8. Download Results


In [ ]:
# Download JSON
files.download(output_json_path)

# Zip and download images if any were extracted
if result["images"]:
    import shutil
    zip_path = os.path.join(OUTPUT_DIR, "extracted_images")
    shutil.make_archive(zip_path, 'zip', extractor.images_dir)
    print(f"✅ Images zipped to: {zip_path}.zip")
    files.download(f"{zip_path}.zip")
else:
    print("No images to download.")
